In [13]:
#help('bdd_sdk')

In [2]:
execfile('ipython/00-bdd-shell-init.py')

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
get_ipython().magic('matplotlib inline')
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import _arma_predict_out_of_sample
from statsmodels.tsa.stattools import acf, pacf, adfuller
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import acf, pacf

In [6]:
dss = bc.datasets()
dss.count

10

In [7]:
for ds in dss:
    print ds


media_demo_customer	default_edp_89c616b6-aa10-4827-aa82-1e9c3fcc419e	default_edp_89c616b6-aa10-4827-aa82-1e9c3fcc419e	Hive	default.media_demo_customer

media_demo_customer	edp_cli_edp_2c7f41ee-65bf-43ac-8bb4-5b6b59a55d75	edp_cli_edp_2c7f41ee-65bf-43ac-8bb4-5b6b59a55d75	Hive	default.media_demo_customer

media_demo_customer	default_edp_c78bef3e-5e71-4ef9-8760-5b23d4a02848	default_edp_c78bef3e-5e71-4ef9-8760-5b23d4a02848	Hive	default.media_demo_customer

media_demo_movielog	default_edp_f343a63a-4cc3-449c-a0a0-d4b50c3d0849	default_edp_f343a63a-4cc3-449c-a0a0-d4b50c3d0849	Hive	default.media_demo_movielog

movie_genre_2	default_edp_d17f911d-eea0-4788-af6f-615f483bc978	default_edp_d17f911d-eea0-4788-af6f-615f483bc978	Hive	default.movie_genre_2

movie_genre	default_edp_7d4c18a5-6f02-4067-9f63-91f950078b1e	default_edp_7d4c18a5-6f02-4067-9f63-91f950078b1e	Hive	default.movie_genre

MT_RLS_RTN	default_edp_db966636-a062-4ed5-8595-7d53cc448d50	default_edp_db966636-a062-4ed5-8595-7d53cc448d50	Hive	d

In [47]:
# Load Requested Opzone List

ds_opzone=dss.dataset('default_edp_25e5118a-e631-4422-8249-666fe77858e4')
#ds.properties()

# Data source to Spark DataFrame to Pandas DataFrame
df_opzone = ds_opzone.to_spark()
pdf_opzone = df_opzone.toPandas()

# Convert to array
#pdf_opzone = pdf_opzone[pdf_opzone.opzone_cde != 'ZHJ']
pdf_opzone = pdf_opzone[pdf_opzone.opzone_cde == 'ZHK']
opzones_list_tmp = pdf_opzone[['opzone_cde']]
opzones_list = opzones_list_tmp['opzone_cde'] 

In [60]:
# Load MT_RLS and MT_RTN Actual List

ds_mt=dss.dataset('default_edp_db966636-a062-4ed5-8595-7d53cc448d50')
#ds.properties()

# Data source to Spark DataFrame to Pandas DataFrame
df_mt = ds_mt.to_spark()
pdf_mt = df_mt.toPandas()

In [61]:
import datetime
def date_convert_fn(x):
    return datetime.datetime.fromtimestamp(x/1000.0).strftime('%Y-%m-%d')

pdf_mt['date'] = pdf_mt.apply(lambda row: date_convert_fn(row['date']), axis=1)

In [62]:
pdf_mt.set_index(keys=['date'], drop=True,inplace=True)
pdf_mt.index=pd.to_datetime(pdf_mt.index)
pdf_mt.head(5)

,PRIMARY_KEY,eqp_size_type,mt_rls,mt_rtn,opzone_cde
date,,,,,
2011-12-31,0-0-0,20GP,26,12,ZAA
2012-01-07,0-0-1,20GP,19,3,ZAA
2012-01-14,0-0-2,20GP,13,7,ZAA
2012-01-21,0-0-3,20GP,10,13,ZAA
2012-01-28,0-0-4,20GP,43,7,ZAA


In [70]:
# ARIMA Time Series
for opzone in opzones_list:
    sizetype = '40GP'
    df = pdf_mt[ (pdf_mt.opzone_cde==opzone) & (pdf_mt.eqp_size_type == sizetype) ]
    df_RLS = df[['mt_rls']]
    df_RTN = df[['mt_rtn']]
    df_RLS.sort_index(inplace=True, ascending=True)
    df_RTN.sort_index( inplace=True, ascending=True)
    df_RLS_TS = df_RLS['mt_rls'] 
    df_RTN_TS = df_RTN['mt_rtn']
    result = pd.DataFrame(columns=('1wk', '2wk', '3wk','4wk', '5wk'))
    
    for n in range(0,53):
        y = df_RTN_TS[0 :-53 + n]

        try:
            res = ARIMA(y, order=(2, 1, 2)).fit(trend="nc")
        except:
            try:
                res = ARIMA(y, order=(2, 1, 1)).fit(trend="nc")
            except:
                try:
                    res = ARIMA(y, order=(1, 1, 2)).fit(trend="nc")
                except:
                    try:
                        res = ARIMA(y, order=(1, 1, 1)).fit(trend="nc")
                    except:
                        print "invertiable error occured."
    
        predication = res.forecast(5)[0]
        result.loc[n] = [predication[0],predication[1],predication[2],predication[3],predication[4]] 
        
    result['OPZONE_CODE'] = opzone
    result['SIZE_TYPE'] = sizetype
    df_temp = pd.DataFrame(df_RTN_TS[-53:])
    result.index = df_temp.index
    df_temp1 = pd.concat([result, df_temp], axis=1)
    df_temp1['2wk'] = df_temp1['2wk'].shift(1)
    df_temp1['3wk'] = df_temp1['2wk'].shift(2)
    df_temp1['4wk'] = df_temp1['2wk'].shift(3)
    df_temp1['5wk'] = df_temp1['2wk'].shift(4)
    df_temp1.head
    #df_temp1.to_csv('./dataout/export_arima_40GP_MTRTN.csv', mode='a', header=False)


In [71]:
df_temp1.head(5)

,1wk,2wk,3wk,4wk,5wk,OPZONE_CODE,SIZE_TYPE,mt_rtn
date,,,,,,,,
2015-09-26,337.748485,NaN,NaN,NaN,NaN,ZHK,40GP,253
2015-10-03,325.323779,339.276865,NaN,NaN,NaN,ZHK,40GP,324
2015-10-10,329.124398,329.339106,NaN,NaN,NaN,ZHK,40GP,326
2015-10-17,323.829790,324.350703,339.276865,NaN,NaN,ZHK,40GP,356
2015-10-24,326.362056,320.983420,329.339106,339.276865,NaN,ZHK,40GP,268


In [72]:
# Convert the output Pandas DataFrame to Spark DataFrame
sdf = sqlContext.createDataFrame(df_temp1)

In [73]:
# Persist the Spark DataFrame into new Hive table
sdf.write.saveAsTable('default.ARIMA_ZHK_40GP_MTRTN')

In [74]:
sdf.printSchema()

root
 |-- 1wk: double (nullable = true)
 |-- 2wk: double (nullable = true)
 |-- 3wk: double (nullable = true)
 |-- 4wk: double (nullable = true)
 |-- 5wk: double (nullable = true)
 |-- OPZONE_CODE: string (nullable = true)
 |-- SIZE_TYPE: string (nullable = true)
 |-- mt_rtn: long (nullable = true)

